# Prototyping a Data Storage Model for ChiantiPy

In [30]:
import os

import numpy as np
import h5py
import astropy.units as u
import ChiantiPy.tools.util as ch_util
import ChiantiPy.core as ch

 using cli
 using CLI for selections
 reading chiantirc file


CHIANTI has several file formats that it stores for each ion. The most notable are,

* `.elvlc`: energy levels (in cm$^{-1}$) with additional level configuration
* `.wgfa`: wavelengths, oscillator strengths, and Einstein A coefficients for the transitions
* `.scups`: temperatures and effective collision strenghts for each transition. Replaces the old `.splups` files. There are also still `.psplups` files
* Additional files:
  * `.fblvl`: information for calculating free-bound continuum
  * `.cilvl`, `.reclvl`: ionization and recombination rates

Essentially, we want to have a property for each of these files. Each of these properties returns an object with a `__getitem__` method that takes in the keys associated with each of these files. These objects return the relevant data streamed out of the HDF5 file.

Ideally, this file would be built once the first time you download ChiantiPy and then only rebuilt when your installed CHIANTI database gets updated. The filename is then stored at the package level. We'll use our CHIANTI database HDF5 file that we've been using in `synthesizAR`.

In [5]:
chianti_hdf5_filename = '/data/datadrive1/ar_forward_modeling/systematic_ar_study/chianti_db.h5'

In [43]:
class DataIndexer(object):
    
    def __init__(self,ion_path,key_conversion):
        self.ion_path = ion_path
        self.key_conversion = key_conversion
    
    def __getitem__(self,key):
        if key not in self.key_conversion:
            raise IndexError('{} not a valid attribute for {}'.format(key,self.ion_path))
        with h5py.File(chianti_hdf5_filename,'r') as hf:
            data = np.array(hf['/'.join([self.ion_path,self.key_conversion[key][0]])])*self.key_conversion[key][1]
        return data
    
    def __repr__(self):
        with h5py.File(chianti_hdf5_filename,'r') as hf:
            ref = hf[self.ion_path].attrs['ref']
        return ref
    

In [44]:
class GenericChiantiData(object):
    
    def __init__(self,ion_name):
        self.ion_name = ion_name
        self.element = ion_name.split('_')[0]
        self.Z = ch_util.el2z(self.element)
        self.stage = ion_name.split('_')[-1]
        
    @property
    def elvlc(self):
        # more sensible and readable keywords
        key_unit_conversion = {'energy':('ecm',1/u.cm),'level':('lvl',u.dimensionless_unscaled)}
        return DataIndexer('/'.join([self.element,self.stage,'elvlc']),key_unit_conversion)
    
    @property
    def wgfa(self):
        key_unit_conversion = {'wavelength':('wvl',u.angstrom),'einstein_a':('avalue',1/u.s)}
        return DataIndexer('/'.join([self.element,self.stage,'wgfa']),key_unit_conversion)
    

In [42]:
with h5py.File(chianti_hdf5_filename,'r') as hf:
    print([a for a in hf['fe/15/scups'].attrs])
    print(hf['fe/15/elvlc'].attrs['ref'])

['ntrans', 'ref']
%filename: fe_15.elvlc
%Experimental energy levels (1 to 45): Shirai,T., Sugar,J., Musgrove,A., Wiese,W.L.,2000, J.Phys.Chem.Ref.Data, Monograph 8
%Experimental energy levels (31,32): Eissner, W., Galavis, M.E., Mendoza, C., Zeippen, C.J., 1999, A&AS, 137, 165
%Experimental energy levels (50,51,52): Reader, J., Sugar, J., 1975, J.Phys.Chem.Ref.Data, 4, 424
%Experimental energy levels (76-117): Ralchenko Y., Jou, F.C., Kelleher, D.E., Kramida, A.E.,
Musgrove, A., Reader, J., Wiese, W.L., Olsen, K., 2005, NIST Atomic Spectra Database, Version 3.0
%Theoretical energy levels: Landi, E. 2011, ADNDT, 97, 587
%comment: NIST energy for level 3p.4f 1F3 (80) has been assigned to level 3p.4f 3F3 (77)
%produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration
%
% Enrico Landi - Aug 2005


In [45]:
chianti_reader = GenericChiantiData('fe_15')

In [47]:
chianti_reader.wgfa

%filename: fe_15.wgfa
%Experimental energy levels (1 to 45): Shirai, T., Sugar, J., Musgrove, A., Wiese, W.L.,2000, J.Phys.Chem.Ref.Data, Monograph 8
%Experimental energy levels (31,32): Eissner, W., Galavis, M.E., Mendoza, C., Zeippen, C.J., 1999, A&AS, 137, 165
%Experimental energy levels (50,51,52): Reader, J., Sugar, J., 1975, J.Phys.Chem.Ref.Data, 4, 424
%Experimental energy levels (76-117): Ralchenko Y., Jou, F.C., Kelleher, D.E., Kramida, A.E.,
Musgrove, A., Reader, J., Wiese, W.L., Olsen, K., 2005, NIST Atomic Spectra Database, Version 3.0
%Theoretical energy levels: Landi, E. 2011, ADNDT, 97, 587
%A values and oscillator strengths: Landi, E. 2011, ADNDT, 97, 587
%A-value (1-3): Traebert, E., Heckmann, P.H., Hutton , R., & Martinson, I. 1988, J. Opt. Soc. Am. B, 5, 2173
%comment: added transition experimental A-value 1-3 from Traebert et al 1988 (E. Landi, Sept. 20, 2006).
%produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration
%
% Enrico Landi -

In [24]:
elvlc_data = chianti_reader.elvlc

In [26]:
elvlc_data['level']

<Quantity [   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
             11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,
             21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,
             31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,
             41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,
             51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,
             61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,
             71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,
             81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,
             91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.,
            101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
            111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
            121., 122., 123., 124., 125., 126., 127., 128., 129., 130.,
            131., 132., 133., 134., 135., 136., 137., 138., 139.

In [27]:
GenericChiantiData('mg_12').elvlc['energy']

<Quantity [        0. , 11870297.1, 11869988.3, 11877615.1, 14070911. ,
            14070818. , 14073078. , 14073074. , 14073825. , 14840837. ,
            14840798. , 14841751. , 14841749. , 14842066. , 14842065. ,
            14842223. , 15197076. , 15197056. , 15197544. , 15197543. ,
            15197705. , 15197705. , 15197786. , 15197786. , 15197834. ] 1 / cm>

In [50]:
%%timeit
GenericChiantiData('h_1').wgfa['einstein_a']

The slowest run took 14.84 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 762 µs per loop


In [51]:
%%timeit
ch.ion('h_1').Wgfa['avalue']

100 loops, best of 3: 13.5 ms per loop


This is specifically for an ion. We could also implement an even more generic class for the other non-ion-specific datasets, e.g. abundance, ionization potential, miscellaneous continuum data. 

Alternatively, when the CHIANTI HDF5 database is created, these could just be broken up by ion appropriately. This would work except for the continuum data which is maybe a special case anyway. 

Basically, we just want to avoid having to index things over and over again. Better to just refer to it by the ion name.

Since this kind of data is used in quite a few places, we could provide it as a generic object. This also makes the CHIANTI data easily accessible without the baggage of the ion object if users want to extend it in anyway.